In [1]:
import os
import re
from dataclasses import dataclass
from time import strftime
from typing import Optional, Dict, Union, Any

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

import datasets
from datasets import Dataset, load_dataset, concatenate_datasets
import evaluate

import torch
import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    DataCollatorForSeq2Seq,
    DataCollatorWithPadding,
    GPTNeoXConfig,
    LlamaConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5Config,
    Trainer,
    TrainingArguments,
    logging as hf_logging,
)
from peft import LoraConfig, TaskType, get_peft_model

chrf_metric = evaluate.load("chrf")


In [ ]:
os.environ['CLEARML_API_ACCESS_KEY'] = ''
os.environ['CLEARML_API_SECRET_KEY'] = ''
os.environ['CLEARML_API_HOST'] = 'https://api.clear.ml'
os.environ['CLEARML_LOG_MODEL'] = 'FALSE'
os.environ['CLEARML_TASK'] = 'en-tatar-post'

In [ ]:
from transformers import AutoTokenizer
import datasets
from datasets import concatenate_datasets

MODEL = 'tencent/HY-MT1.5-7B'

tokenizer = AutoTokenizer.from_pretrained(MODEL)

def build_text_en_tt(example):
    """
    Format English->Chuvash training examples using HY-MT1.5 chat template.
    """
    source_text = example['en']
    target_text = example['tt']
    
    messages = [
        {
            "role": "user",
            "content": f"Translate the following segment into Tatar, without additional explanation.\n\n{source_text}"
        },
        {
            "role": "assistant",
            "content": target_text
        }
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    example["text"] = text
    return example

def build_text_en_tt(example):
    """
    Format Russian->Chuvash training examples using HY-MT1.5 chat template.
    """
    source_text = example['en']
    target_text = example['tt']
    
    messages = [
        {
            "role": "user",
            "content": f"Translate the following segment into Chuvash, without additional explanation.\n\n{source_text}"
        },
        {
            "role": "assistant",
            "content": target_text
        }
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    example["text"] = text
    return example

def tokenize_example(example):
    """
    Tokenize the formatted text for training.
    """
    encoded = tokenizer(
        example["text"],
        truncation=True,
        max_length=512,
        padding=False
    )
    example["input_ids"] = encoded["input_ids"]
    example["attention_mask"] = encoded["attention_mask"]
    
    example["labels"] = encoded["input_ids"].copy()
    
    return example

en_tt_full = datasets.load_dataset('Defetya/turkish-langs-ru-en-mt', 'en-tt')['train']


en_tt_split = en_tt_full.train_test_split(test_size=0.05, seed=42)
en_tt_train = en_tt_split['train']
en_tt_test = en_tt_split['test']

en_tt_train = en_tt_train.map(build_text_en_tt, num_proc=64)
en_tt_train = en_tt_train.map(tokenize_example, batched=False, num_proc=64)

en_tt_test = en_tt_test.map(build_text_en_tt, num_proc=8)
en_tt_test = en_tt_test.map(tokenize_example, batched=False, num_proc=16)

print(f"EN-CHV train: {len(en_tt_train)} examples")
print(f"EN-CHV test: {len(en_tt_test)} examples")


ds = datasets.DatasetDict({
    'train': en_tt_train,
    'test': en_tt_test
})



ds['test'] = ds['test'].select(range(100))

print(f"\nTrain: {len(ds['train'])} examples (EN+RU -> TT, shuffled)")
print(f"Test: {len(ds['test'])} examples (EN -> TT)")

EN-CHV train: 295463 examples
EN-CHV test: 15551 examples

Train: 295463 examples (EN+RU -> CHV, shuffled)
Test: 100 examples (EN -> CHV)


In [ ]:
ds['train'] = ds['train'].remove_columns(['tt', 'en', 'text'])
ds['test'] = ds['test'].remove_columns(['tt', 'en', 'text'])

In [ ]:
MODEL = 'tencent/HY-MT1.5-7B'

tokenizer = AutoTokenizer.from_pretrained(MODEL, padding_side='left')

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    MODEL, 
    dtype=torch.bfloat16, 
).cuda()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

target_modules = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
]

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=target_modules,
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 13,631,488 || all params: 7,518,199,808 || trainable%: 0.1813


In [ ]:
IS_7B = True
if IS_7B:
    ASSISTANT_TOKEN = 127962
else:
    ASSISTANT_TOKEN = 120007

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    # 2) If preds are logits: (bs, seq, vocab) -> take argmax
    if isinstance(preds, np.ndarray) and preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    # 3) Replace -100 (or any negative ids) in BOTH preds and labels
    preds = np.where(preds < 0, tokenizer.pad_token_id, preds)
    labels = np.where(labels < 0, tokenizer.pad_token_id, labels)

    # 4) Tokenizers like int32 / int64; make it explicit
    preds = preds.astype(np.int32)
    labels = labels.astype(np.int32)

    # 5) Slice to keep only tokens AFTER assistant token (not including it)
    truncated_preds = []
    truncated_labels = []
    
    for pred_seq, label_seq in zip(preds, labels):
        pred_asst_pos = np.where(pred_seq == ASSISTANT_TOKEN)[0]
        if len(pred_asst_pos) > 0:
            start_pos = pred_asst_pos[0] + 1
            truncated_preds.append(pred_seq[start_pos:])
        else:
            truncated_preds.append(pred_seq)
        
        label_asst_pos = np.where(label_seq == ASSISTANT_TOKEN)[0]
        if len(label_asst_pos) > 0:
            start_pos = label_asst_pos[0] + 1
            truncated_labels.append(label_seq[start_pos:])
        else:
            truncated_labels.append(label_seq)

    decoded_preds = tokenizer.batch_decode(truncated_preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(truncated_labels, skip_special_tokens=True)

    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]
    
    print(f'Generated (first 3): {decoded_preds[:10]}')
    print(f'Labels (first 3): {decoded_labels[:10]}')
    
    result = chrf_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        word_order=2
    )
    return {"chrf": result["score"]}


class HYMTSeq2SeqTrainer(Seq2SeqTrainer):

    def _mask_prompt_labels(self, inputs):
        if "labels" not in inputs or "input_ids" not in inputs:
            return inputs

        input_ids = inputs["input_ids"]
        labels = inputs["labels"]
        if not torch.is_tensor(labels):
            return inputs

        labels = labels.clone()

        for i in range(input_ids.size(0)):
            pos = (input_ids[i] == ASSISTANT_TOKEN).nonzero(as_tuple=True)[0]
            if pos.numel() == 0:
                labels[i, :] = -100
            else:
                cut = int(pos[0].item()) + 1
                labels[i, :cut] = -100

        inputs["labels"] = labels
        return inputs

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        inputs = self._mask_prompt_labels(inputs)
        return super().compute_loss(model, inputs, return_outputs=return_outputs, **kwargs)

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None, **gen_kwargs):
        has_labels = "labels" in inputs
        inputs = self._prepare_inputs(inputs)

        if has_labels:
            inputs = self._mask_prompt_labels(inputs)

        with torch.no_grad():
            if has_labels:
                loss, _ = self.compute_loss(model, inputs, return_outputs=True)
                loss = loss.mean().detach()
            else:
                loss = None

        if prediction_loss_only:
            return (loss, None, None)

        if self.args.predict_with_generate:
            ASSISTANT_TOKEN = 120007
            input_ids = inputs["input_ids"]
            batch_size = input_ids.shape[0]

            truncated_ids = []
            truncated_masks = []

            for i in range(batch_size):
                seq = input_ids[i]
                pos = (seq == ASSISTANT_TOKEN).nonzero(as_tuple=True)[0]
                if pos.numel() > 0:
                    cut = int(pos[0].item()) + 1
                    seq = seq[:cut]
                truncated_ids.append(seq)
                truncated_masks.append(torch.ones_like(seq))

            max_len = max(x.numel() for x in truncated_ids)
            pad_token_id = self.processing_class.pad_token_id

            padded_ids = []
            padded_masks = []
            for ids, mask in zip(truncated_ids, truncated_masks):
                pad_len = max_len - ids.numel()
                if pad_len > 0:
                    ids = torch.cat([ids, torch.full((pad_len,), pad_token_id, dtype=ids.dtype, device=ids.device)])
                    mask = torch.cat([mask, torch.zeros(pad_len, dtype=mask.dtype, device=mask.device)])
                padded_ids.append(ids)
                padded_masks.append(mask)

            gru_inputs = {
                "input_ids": torch.stack(padded_ids),
                "attention_mask": torch.stack(padded_masks),
            }

            gen_kwargs = self._gen_kwargs.copy()
            gen_kwargs = dict(
                max_new_tokens=512,
                num_beams=1,
                do_sample=True,
                num_return_sequences=1,
                repetition_penalty=1.1,
                early_stopping=True,
                temperature=0.7,
                pad_token_id=tokenizer.pad_token_id,
            )
            if gen_kwargs.get("max_length") is None and gen_kwargs.get("max_new_tokens") is None:
                gen_kwargs["max_new_tokens"] = self.args.generation_max_length
            if gen_kwargs.get("num_beams") is None:
                gen_kwargs["num_beams"] = self.args.generation_num_beams

            generated_tokens = model.module.generate(**gru_inputs, **gen_kwargs)
        else:
            generated_tokens = None

        labels = inputs["labels"] if has_labels else None
        return (loss, generated_tokens, labels)



data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8,
    label_pad_token_id=-100
)

training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints_7B_lora_translated/en-chv-final",

    num_train_epochs=2,
    per_device_train_batch_size=4,  # Adjust based on GPU memory
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,

    # Learning rate
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    # Optimizer
    optim="adamw_torch",
    weight_decay=0.01,

    # Gradient handling
    max_grad_norm=1.0,

    # Logging
    logging_steps=10,
    logging_dir="./logs",
    logging_strategy="steps",

    # Evaluation
    eval_strategy="steps",
    eval_steps=1_000,

    # Generation for chrf++
    predict_with_generate=True,
    generation_max_length=512,  # Max tokens to generate
    generation_num_beams=5,  # Beam search for better quality

    # Saving
    save_strategy="steps",
    save_steps=1_000,
    save_total_limit=1,
    save_only_model=True,
    save_safetensors=False,
    load_best_model_at_end=False,
    metric_for_best_model="chrf",  # Use chrF++ for best model
    greater_is_better=True,  # Higher chrF is better

    dataloader_num_workers=4,
    dataloader_pin_memory=True,

    # Reproducibility
    seed=42,

    # Reporting
    report_to="clearml",

    remove_unused_columns=True,
    push_to_hub=False,
)


trainer = HYMTSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 127958}.


ClearML Task: overwriting (reusing) task id=5bb40475684b47e2a72ec7dcc1ad6111
2026-01-11 19:03:56,680 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/c26a6bbc77a549af84da889d0ac9231f/experiments/5bb40475684b47e2a72ec7dcc1ad6111/output/log
2026-01-11 19:03:59,954 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


Step,Training Loss,Validation Loss,Chrf
1000,0.711500,0.865167,82.120361
2000,0.537100,0.675481,79.797213
3000,0.516100,0.606926,79.204580
4000,0.422000,0.547555,82.950353
5000,0.421800,0.505054,84.330510
6000,0.420000,0.497156,81.378150
7000,0.395400,0.471256,81.251235
8000,0.360600,0.448247,85.432927
9000,0.347400,0.436362,85.961672
10000,0.341300,0.422246,83.603843


/home/jovyan/.mlspace/envs/bolgov_simson_training/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated (first 3): ['Дәхи Раббың сине пәйгамбәр булганчы хак диннән адашучы табып, аннары туры юлга күндерде түгелме?Инде Аллаһ китапта язылган рөхсәт исеменең җавапын анык күрсәтеп бирер.', 'Менә сез чокырның Мәдинәгә якын тарафында булдыгыз, әмма кәферләр чокырның Мәккә тарафында булдылар, кәрван сездән түбән иде.Ий! Кеше телеге миңа хөснәт булсын, вә ачык кабылкая җыелып торган эшләрне билеп китсен, әмер итеп бирсен.', 'Дәхи болытлардан бере артыннан бере коелучы яңгыр суын иңдердек.Шайтер кирәк үзен тез урынына китергән, бу шайтер бозык ясалмады.', 'Рәсүлләр 2: 38 - 42; 4: 32 - 37; 5: 1 - 11Шулай ук моны билгән булырга мөмкин, алар китапларда һәм язмалардагы хакыйкатьне бозтырмау өчен, дәхи шундан кере торы хакыйкатьне белеп, алар сезнең атагыз Яхъянең генә Аллаһыдан чыган иң яңы Рәсүле иң ашкын рәвештә эшлә.', 'Әмма яңадан терелгән майланганнарның даны аларның даныннан күпкә зуррак.Шулай ук Хезмәтче Моисей бу эшне дөрес әйткән.', 'Нацистлар Германиясендә, Анголада, Малавида, Моз

...

Generated (first 3): ['Дәхи Раббың сине пәйгамбәр булганчы хак диннән адашучы табып, аннары туры юлга күндерде түгелме?Ияле милкенеҙе үзегез әзаларыгыз, җир, күкләр, уңуда, сулу, яку, яныч и, эс, нәфес вә күп т тәүбә ителгән зикрәләрне.', 'Менә сез чокырның Мәдинәгә якын тарафында булдыгыз, әмма кәферләр чокырның Мәккә тарафында булдылар, кәрван сездән түбән иде.Шулай булса да, сабыр чаплыйсыз, аларныґа милтә кирәк җибәрегез, дип хәрам ителгән түгел.', 'Дәхи болытлардан бере артыннан бере коелучы яңгыр суын иңдердек.Яшил үскенчеге чиркәве турында нәрсә дип уйлап китин?', 'Рәсүлләр 2: 38 - 42; 4: 32 - 37; 5: 1 - 11Англия.', 'Әмма яңадан терелгән майланганнарның даны аларның даныннан күпкә зуррак.Аллаһу тәгаләнең исеме хак ун б м миллион доллар США', 'Нацистлар Германиясендә, Анголада, Малавида, Мозамбикта һәм башка күп илләрдә хакимият башлыклары Йәһвә Шаһитләрен юк итәр өчен көч куйганнар.Яшь укучылар арасында Йәһвә Шаһитләре буларак тапкан максатларны назура алдымы?', 'Шулай булса, ул

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 73dd7a85-0848-434d-8e36-27c7ea4e3aa5)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].


Generated (first 3): ['Дәхи Раббың сине пәйгамбәр булганчы хак диннән адашучы табып, аннары туры юлга күндерде түгелме?Иманлы муслиманлар җитәрлекдән зур рәхмәткә лаик, Аллаһу тәґаләнең рәхмәтен шәкертләренә түгелме?', 'Менә сез чокырның Мәдинәгә якын тарафында булдыгыз, әмма кәферләр чокырның Мәккә тарафында булдылар, кәрван сездән түбән иде.Хатиб Ибраһим асасына мондый гәүдәләрне йөкләп, нинди фадихәт бүләгем! диде.', 'Дәхи болытлардан бере артыннан бере коелучы яңгыр суын иңдердек.Авторлар: Мәсьел Алиев', 'Рәсүлләр 2: 38 - 42; 4: 32 - 37; 5: 1 - 11Шаушуннан килгән сору', 'Әмма яңадан терелгән майланганнарның даны аларның даныннан күпкә зуррак.Шулай ук ни өчен?', 'Нацистлар Германиясендә, Анголада, Малавида, Мозамбикта һәм башка күп илләрдә хакимият башлыклары Йәһвә Шаһитләрен юк итәр өчен көч куйганнар.Комментарий.', 'Шулай булса, ул бу сәгать эчендә аның өчен эшләгәнегезне кадерли икәненә шикләнмәгез.Компьютер таратылып куртластыралма.', 'Ничек бу кеше Йәһвә турында белгән һәм Теок

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 33341d74-7e4e-48ab-9478-f6cdedb91f6b)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 2af77634-51c6-4865-9eac-a4a6928e22de)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].


TrainOutput(global_step=36934, training_loss=0.3316070172604287, metrics={'train_runtime': 28243.7217, 'train_samples_per_second': 20.922, 'train_steps_per_second': 1.308, 'total_flos': 6.927392410800292e+18, 'train_loss': 0.3316070172604287, 'epoch': 2.0})

In [10]:
tokenizer.save_pretrained('checkpoints_7B_lora_translated/tatar_final_lora')

('checkpoints_7B_lora_translated/tatar_final_lora/tokenizer_config.json',
 'checkpoints_7B_lora_translated/tatar_final_lora/special_tokens_map.json',
 'checkpoints_7B_lora_translated/tatar_final_lora/chat_template.jinja',
 'checkpoints_7B_lora_translated/tatar_final_lora/tokenizer.json')

In [9]:
trainer.model.save_pretrained('checkpoints_7B_lora_translated/tatar_final_lora')

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 83b06db4-5513-4c6d-baa0-8e5a3b0ece78)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].
